In [1]:
%matplotlib inline

# Important: We are using PIL to read .png files later.
# This was done on purpose to read indexed png files
# in a special way -- only indexes and not map the indexes
# to actual rgb values. This is specific to PASCAL VOC
# dataset data. If you don't want thit type of behaviour
# consider using skimage.io.imread()
from PIL import Image
import numpy as np
import skimage.io as io
import tensorflow as tf

# pytorch
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [2]:
train = pd.read_csv("../asset/train.csv")

In [3]:
train.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [10]:
train["Path"] = ["../asset/train/" + filename for filename in train["Image"]]

In [11]:
train.head()

,Image,Id,Path
0,0000e88ab.jpg,w_f48451c,../asset/train/0000e88ab.jpg
1,0001f9222.jpg,w_c3d896a,../asset/train/0001f9222.jpg
2,00029d126.jpg,w_20df2c5,../asset/train/00029d126.jpg
3,00050a15a.jpg,new_whale,../asset/train/00050a15a.jpg
4,0005c1ef8.jpg,new_whale,../asset/train/0005c1ef8.jpg


## As you can see, it's an imbalanced dataset

In [12]:
train.Id.value_counts().head()

new_whale    9664
w_23a388d      73
w_9b5109b      65
w_9c506f6      62
w_0369a5c      61
Name: Id, dtype: int64

In [13]:
df = train[train['Id'] != 'new_whale']
df.Id.value_counts().head()

w_23a388d    73
w_9b5109b    65
w_9c506f6    62
w_0369a5c    61
w_700ebb4    57
Name: Id, dtype: int64

In [6]:
len(train['Id'].unique())

5005

In [14]:
unique_classes = pd.unique(df['Id'])
encoding = dict(enumerate(unique_classes))
encoding = {value: key for key, value in encoding.items()}
df = df.replace(encoding)

In [15]:
df.head()

,Image,Id,Path
0,0000e88ab.jpg,0,../asset/train/0000e88ab.jpg
1,0001f9222.jpg,1,../asset/train/0001f9222.jpg
2,00029d126.jpg,2,../asset/train/00029d126.jpg
6,000a6daec.jpg,3,../asset/train/000a6daec.jpg
8,0016b897a.jpg,4,../asset/train/0016b897a.jpg


In [17]:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)

preprocess = transforms.Compose([
   transforms.Grayscale(num_output_channels=1),
   transforms.Resize(128),
   transforms.CenterCrop(128),
   transforms.ToTensor(),
   normalize
])

# imgs = [Image.open(whale) for whale in full_path_random_whales]
# imgs_tensor = [preprocess(whale) for whale in imgs]

In [19]:
test = df['Path'][:1000]
imgs = [Image.open(whale) for whale in test]
imgs_tensor = torch.stack([preprocess(whale) for whale in imgs])